In [ ]:
from scripts.data_util.DataTransmitter import *
from scripts.data_util.DataUpdater import *

Updater = DataUpdater(False)
Updater.update_something(KEY = ['trade_Xday'])
Updater.print_current_unfetch()
Updater.dump_current_updater()

In [2]:
from scripts.data_util.DataTransmitter import *
from scripts.data_util.DataUpdater import *

get_trade_Xday(19970122 , 5)

d:\Anaconda3\lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
d:\Anaconda3\lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


<class 'scripts.data_util.DataTank.Data1D'>
secid len (517): array([     1,      2,      3,      4,      5,      6,      7,      8,
            9,     10,     11,     12,     13,     14,     15,     16,
           17,     18,     19,     20,     21,     22,     23,     24,
           25,     26,     27,     28,     29,     30,     31,     32,
           33,     34,     35,     36,     37,     38,     39,     40,
           42,     43,     45,     46,     47,     48,     49,     50,
           55,     56,     58,     61,    401,    402,    403,    404,
          405,    406,    407,    408,    409,    410,    411,    412,
          413,    415,    416,    417,    419,    420,    421,    422,
          423,    425,    426,    428,    430,    501,    502,    503,
          504,    505,    506,    507,    508,    509,    510,    511,
          513,    514,    515,    516,    517,    518,    519,    520,
          521,    522,    523,    524,    525,    526,    527,    528,
          529,  

In [ ]:
date = 19970122
x_day = 5
tol = 1e-8
db_path_info = f'{DIR_data}/DB_data/DB_information/DB_information.h5'
with DataTank(db_path_info , 'r') as info:
    calendar = info.read_dataframe('basic/calendar')
rolling_dates = calendar.calendar[calendar.trade > 0].to_numpy().astype(int)
rolling_dates = sorted(rolling_dates[rolling_dates <= int(date)])[-x_day:]
assert rolling_dates[-1] == date , (rolling_dates[-1] , date)
groups = np.array(rolling_dates).astype(int) // 10000

db_path_day = DIR_data + '/DB_data/DB_trade_day/DB_trade_day.{}.h5'
src_files = [db_path_day.format(group) for group in groups]
if not all([os.path.exists(file) for file in src_files]): return unfetched_data()

price_feat  = ['open','close','high','low','vwap']
volume_feat = ['amount','volume','turn_tt','turn_fl','turn_fr']
pctchg_feat = ['pctchange']
all_feat = price_feat + volume_feat + pctchg_feat

dtank , datas , secid = DataTank() , [] , None
for d , src in zip(rolling_dates , src_files):
    src_path , inner_path = os.path.join(db_path_day , src) , f'/day/trade/{d}'
    if dtank.filename != src_path: 
        dtank.close()
        dtank = DataTank(src_path , 'r')
    if dtank.get_object(inner_path) is None:
        data = get_trade_day(date , tol = tol , **kwargs)
    else:
        data = dtank.read_data1D(inner_path)
    if isinstance(data , unfetched_data) or data is None:
        return unfetched_data()
    secid = data.secid if secid is None else np.intersect1d(secid , data.secid)
    datas.append(data)
dtank.close()

for i , data in enumerate(datas):
    data = data.to_dataframe().loc[secid]
    data.loc[:,price_feat] = data.loc[:,price_feat] * data.loc[:,'adjfactor'].values[:,None]
    datas[i] = data.loc[:,all_feat].values

data = np.stack([data for data in datas] , axis = 0)
del datas

df = pd.DataFrame({'secid':secid})
# price_feat
df['open']   = data[...,all_feat.index('open')][0]
df['close']  = data[...,all_feat.index('close')][-1]
df['high']   = data[...,all_feat.index('high')].max(axis = 0)
df['low']    = data[...,all_feat.index('low')].min(axis = 0)

for feat in volume_feat: df[feat] = data[...,all_feat.index(feat)].sum(axis = 0)

df['vwap']      = (data[...,all_feat.index('vwap')]*data[...,all_feat.index('volume')]).sum(axis = 0) / df['volume']
df['pctchange'] = (data[...,all_feat.index('pctchange')] / 100 + 1).prod(axis = 0) * 100 - 100


In [16]:
type(a)

scripts.data_util.DataTransmitter.unfetched_data

In [1]:
%run data_processing.py
# memory leaking!

23-12-27 02:03:38|MOD:data_processing|: Data Processing start!
23-12-27 02:03:38|MOD:data_processing|: 3 datas :['y', 'trade_day', 'trade_15m']


Wed Dec 27 02:03:38 2023 : y start ...
y blocks reading 10days/lag1 Data1D's ...... cost 9.05 secs
y blocks reading 20days/lag1 Data1D's ...... cost 8.39 secs
y blocks merging ...... cost 4.10 secs
y blocks process ...... cost 0.55 secs
y blocks saving  ...... cost 18.51 secs
y blocks norming ...... cost 0.00 secs
Wed Dec 27 02:04:19 2023 : y finished! Cost 40.67 Seconds
Wed Dec 27 02:04:19 2023 : trade_day start ...
trade_day blocks reading day/trade Data1D's ...... cost 46.29 secs
trade_day blocks merging ...... cost 0.12 secs
trade_day blocks process ...... cost 5.34 secs
trade_day blocks saving  ...... cost 25.05 secs
trade_day blocks norming ...... cost 34.22 secs
Wed Dec 27 02:06:10 2023 : trade_day finished! Cost 111.11 Seconds
Wed Dec 27 02:06:10 2023 : trade_15m start ...
trade_15m blocks reading 15min/trade Data1D's ...... cost 1045.64 secs
trade_15m blocks merging ...... cost 0.11 secs
trade_15m blocks process ...... cost 16.26 secs
trade_15m blocks saving  ...... cost 166.6

23-12-27 02:27:22|MOD:data_processing|: Data Processing Finished! Cost 1423.68 Seconds


... cost 42.68 secs
Wed Dec 27 02:27:21 2023 : trade_15m finished! Cost 1271.34 Seconds
